In [1]:
from matplotlib import pyplot as plt
import numpy as np
from einops import rearrange 
import os
log_path = './log/stocks'

In [24]:
stocks = ['US', 'EU', 'HK']
algs = ['S4'] # Mega is not ready yet
original_stocks = []
scaled_stocks = []
observed_masks = []
for ticker in stocks:
    scaled_stock = np.load('datasets/Stocks/scaled_' + ticker + "_all_stocks_2018-01-02_to_2023-01-01.npy", allow_pickle=True)
    scaled_stock = np.nan_to_num(scaled_stock.astype(float)) # L N K 
    scaled_stocks.append(scaled_stock)
    original_stock = np.load('datasets/Stocks/' + ticker + "_all_stocks_2018-01-02_to_2023-01-01.npy", allow_pickle=True)
    original_stocks.append(original_stock.astype(float)) # L N K 
    observed_mask = (~np.isnan(original_stock.astype(float))).astype(float) # L N K 
    observed_masks.append(observed_mask)
    
original_stocks = np.concatenate(original_stocks, axis=1)[305:] # L 3N K
scaled_stocks = np.concatenate(scaled_stocks, axis=1)[305:]
observed_masks = np.concatenate(observed_masks, axis=1)[305:]

for alg in algs:
    log_path = 'log/stocks/all/SSSD-'+ alg + "/" 
    past_time = "20230131-093821_T_100_Layers_20"
    mask_path = log_path + past_time + '/gt_mask.npy'
    gt_mask = np.load(mask_path).astype(bool).transpose([2, 0, 1]) # L N K
    target_mask = observed_masks - gt_mask
    generated_imputation = np.load('results/stocks/all/SSSD-'+alg+"/" +past_time+'/generated_samples/imputed_all_data.npy')
    gs_median = np.median(generated_imputation, axis = 0).transpose([2, 0, 1])# L N K
    gs = scaled_stocks.copy()
    gs[target_mask.astype(bool)] = gs_median[target_mask.astype(bool)]
    mse = np.sum(((gs_median - scaled_stocks)*target_mask)**2)/np.sum(target_mask)
    ase = np.sum(abs(gs[target_mask.astype(bool)] - scaled_stocks[target_mask.astype(bool)]))/np.sum(target_mask)
    print("The mse:{} and ase:{}".format(mse, ase))

        
        
        
        
        

The mse:0.01599550219920305 and ase:0.10365643034339295


In [21]:
gs_median.shape


(1000, 1144, 6)

In [37]:
res = gs[target_mask.astype(bool)] - scaled_stock[target_mask.astype(bool)]

In [59]:
targets=(gs_median - scaled_stock)*target_mask

In [64]:
np.where(np.sum(np.sum(target_mask, axis=2), 1)!=0)

(array([  20,   32,   62,   72,   85,  108,  127,  131,  150,  168,  186,
         198,  223,  232,  243,  268,  280,  292,  314,  327,  348,  353,
         372,  397,  401,  417,  447,  461,  466,  486,  497,  515,  542,
         569,  580,  606,  612,  629,  659,  685,  691,  713,  721,  738,
         755,  768,  799,  804,  817,  832,  860,  874,  888,  906,  926,
         939,  957,  969,  978,  994, 1015, 1032, 1041, 1061, 1084, 1090,
        1117, 1124, 1149, 1154, 1174, 1189, 1204, 1219, 1235, 1259, 1270,
        1285, 1311, 1321, 1349, 1365, 1382, 1416, 1439, 1447, 1456, 1479,
        1497, 1518, 1527, 1540, 1563, 1571, 1587, 1606, 1628, 1638, 1654,
        1673, 1695, 1707, 1723, 1729, 1747, 1781, 1794, 1819, 1833, 1841,
        1858, 1887, 1889, 1904, 1923, 1946, 1958, 1968, 1992, 2006, 2017,
        2038, 2054, 2075, 2093, 2107, 2121, 2135, 2145, 2174, 2184, 2193,
        2216, 2231, 2255, 2266, 2285, 2290, 2311, 2329, 2345, 2356, 2383,
        2399, 2402, 2419, 2444, 2461, 

In [66]:
gs_median[20,0]

array([-0.39841458, -0.13024904, -0.04931025, -0.34036946, -0.14841115,
        0.07395928, -0.00334443, -0.09356871, -0.09546655, -0.14681196,
       -0.25515264, -0.24752036, -0.07043433, -0.18256807, -0.08656053,
       -0.21981406,  0.03142746, -0.04086707, -0.07836024, -0.03994777,
       -0.04215567, -0.23038797, -0.15663794, -0.02648901,  0.05195946,
       -0.07211755, -0.14367148, -0.18976265, -0.02623729, -0.14948072,
       -0.06329624, -0.11395498, -0.10189386, -0.02880776, -0.12114738,
        0.1010138 , -0.04222931,  0.09391417, -0.06045301,  0.29960638,
        0.08563817, -0.08509768, -0.09819707, -0.05604534, -0.02747214,
        0.0272062 , -0.06490008, -0.08530415,  0.11118132,  0.01138204,
       -0.07822417, -0.01639464, -0.06378565,  0.05364446,  0.03497181,
       -0.07931484,  0.0960203 ], dtype=float32)

In [67]:
scaled_stock[20,0]


array([ 0.87408084, -0.51895048, -0.73102824,  0.75555569, -0.95570569,
        0.80434751, -0.4311164 , -0.00819691,  0.9076682 , -0.53342283,
        0.55555533, -0.68193863, -0.18577124,  0.19083958, -0.52889937,
       -0.45251859, -0.4354341 ,  0.94000015,  2.02217294,  2.29791678,
        0.18655584, -0.89758105, -0.20748763, -0.17891377, -0.35086709,
        0.18843447, -0.74580835, -1.09064324, -0.58589644, -1.22029862,
        0.42441839, -0.04119978,  0.20591231,  0.65036452, -0.55946221,
       -0.98108164,  0.49328867,  1.43178962, -0.53834472, -0.17995206,
        1.42783535,  0.2607973 , -0.45657009,  0.43620187,  0.44528543,
        2.26046517, -1.02506391, -0.65416564, -0.68319766, -0.66851346,
        0.28367112,  0.78840593, -0.46889542, -0.73302994, -0.6503068 ,
       -0.09318447, -0.64977564])

In [27]:
scaled_stock[~gt_mask].mean()

-0.03304569257821389

In [45]:
np.sum(target_mask)/57/6

157.0

In [58]:
target_mask

53694.0

In [56]:
gt_mask[13]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  T

In [54]:
observed_masks[13]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
 